In [ ]:
#Se conecta Google Colaboratory con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Instala pyspark en Google Colaboratory
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=6327efbb2acc22670c88083a2f0584ee487e6e58eb60e4aee273a5a148a7cefa
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import re
import os # Proporciona funciones para interactuar con el sistema operativo.
import requests # Se utiliza para realizar solicitudes HTTP.
import pandas as pd # Una librería de análisis de datos.
import seaborn as sns #S e utiliza para la visualización de datos.
import pyspark.pandas as ps # Proporciona una interfaz para trabajar con datos en Spark utilizando el formato de DataFrame de pandas.
import json # Se utiliza para trabajar con datos en formato JSON.
from pyspark.sql import SparkSession # Se utiliza para crear una instancia de SparkSession, que es la entrada principal para trabajar con Spark SQL.
from pyspark.sql import functions as F #  Proporciona funciones para trabajar con datos en Spark DataFrame.
from pyspark.sql.functions import array_contains # Esta función se utiliza para filtrar los datos basados en la presencia de un valor en un array.
from pyspark.sql.functions import sum, col # Se utiliza para acceder a una columna en un DataFrame de Spark.
from pyspark.sql.functions import split, substring, concat_ws
from pyspark.sql.functions import expr
from pyspark.sql.types import StringType

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("meta-modificaciones")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [ ]:
# Ruta a la carpeta que contiene los archivos CSV en Google Drive
carpeta = '/content/drive/MyDrive/Colab-Notebooks/Dataset-Limpios/metadata-limpio.csv/'

# Listar archivos en la carpeta
archivos = os.listdir(carpeta)

# Crear una lista para almacenar los DataFrames
df_list = []

# Iterar sobre los archivos y cargar cada archivo CSV en un DataFrame
for archivo_name in archivos:
    if archivo_name.endswith('.csv'):
        archivo_path = os.path.join(carpeta, archivo_name)
        df = spark.read.csv(archivo_path, header=True)
        # Convertir la columna "MISC" a tipo String si es necesario
        if 'MISC' in df.columns:
            df = df.withColumn("MISC", col("MISC").cast("string"))
        df_list.append(df)

# Unir los DataFrames en uno solo
df_final = df_list[0]
for dataframe in df_list[1:]:
    df_final = df_final.unionByName(dataframe)

# Mostrar el DataFrame final
df_final.show()

+--------------------+----------+--------------------+--------------------+------------------+------------------+--------------------+--------------+--------------------+-----+-------------+
|             address|avg_rating|            category|             gmap_id|          latitude|         longitude|                name|num_of_reviews|                 url|state|         city|
+--------------------+----------+--------------------+--------------------+------------------+------------------+--------------------+--------------+--------------------+-----+-------------+
|Char's Hot Dogs, ...|       4.7|Hot dog restauran...|0x153326668aff359...|        38.0461618|      -122.1616748|     Char's Hot Dogs|            38|https://www.googl...|   CA|      Benicia|
|New York Kim-Bob ...|       4.3|Korean restaurant...|0x4065fd476208a27...|40.764643199999995|-73.81192399999999|New York Kim-Bob ...|            68|https://www.googl...|   NY|       Queens|
|Rob's Lunch Box D...|       4.8|Restaurant, 

In [ ]:
df_final.count()

NameError: name 'df_final' is not defined

In [ ]:
def limpiar_lat_lon(lat_lon):
    # Utilizar una expresión regular para extraer solo los números y el punto decimal
    cleaned_lat_lon = re.findall(r'-?\d+\.\d{6}', lat_lon)
    if cleaned_lat_lon:
        return cleaned_lat_lon[0]
    return None

In [ ]:
# Aplicar la función de limpieza a las columnas de latitud y longitud
limpiar_lat_lon_udf = F.udf(limpiar_lat_lon)
df_final = df_final.withColumn("latitude", limpiar_lat_lon_udf("latitude"))
df_final = df_final.withColumn("longitude", limpiar_lat_lon_udf("longitude"))

In [ ]:
df_final.show()

+--------------------+----------+--------------------+--------------------+---------+-----------+--------------------+--------------+--------------------+-----+------------+
|             address|avg_rating|            category|             gmap_id| latitude|  longitude|                name|num_of_reviews|                 url|state|        city|
+--------------------+----------+--------------------+--------------------+---------+-----------+--------------------+--------------+--------------------+-----+------------+
|Tacos MX, 312 W 7...|       4.4|  Mexican restaurant|0x14e1b0df6d819da...|34.045377|-118.253996|            Tacos MX|            48|https://www.googl...|   CA| Los Angeles|
|Honey's Amherst, ...|       2.3|Restaurant, Sport...|0x405bc8c8e897cb6...|42.991173| -78.803115|     Honey's Amherst|             8|https://www.googl...|   NY|     Amherst|
|Stewart's Shops, ...|       3.9|Gas station, Alte...|0x4cca3929c127773...|44.726604| -73.436000|     Stewart's Shops|            

In [ ]:
# Ruta al archivo CSV local
file_path = '/content/drive/MyDrive/Colab-Notebooks/Datos-Limpios/metadata-limpio.csv'

# Escribe el DataFrame a un archivo CSV localmente sin incluir el índice
df.write.csv(file_path, header=True, mode='overwrite')

# **ESTADOS LIMPIOS**

In [ ]:
# Ruta a la carpeta que contiene los archivos CSV en Google Drive
carpeta = '/content/drive/MyDrive/Colab-Notebooks/Dataset-Limpios/estados-limpios.csv'

# Listar archivos en la carpeta
archivos = os.listdir(carpeta)

# Crear una lista para almacenar los DataFrames
df_list = []

# Iterar sobre los archivos y cargar cada archivo CSV en un DataFrame
for archivo_name in archivos:
    if archivo_name.endswith('.csv'):
        archivo_path = os.path.join(carpeta, archivo_name)
        df = spark.read.csv(archivo_path, header=True)
        # Convertir la columna "MISC" a tipo String si es necesario
        if 'MISC' in df.columns:
            df = df.withColumn("MISC", col("MISC").cast("string"))
        df_list.append(df)

# Unir los DataFrames en uno solo
df_estados = df_list[0]
for dataframe in df_list[1:]:
    df_estados = df_estados.unionByName(dataframe)

# Mostrar el DataFrame final
df_estados.show()

+--------------------+------+--------------------+--------------------+--------------------+
|             gmap_id|rating|                text|                time|             user_id|
+--------------------+------+--------------------+--------------------+--------------------+
|0x89de09b4a848543...|     3|Seems to be going...|2018-04-06T00:56:...|11294936078038757...|
|0x89de09b4a848543...|     4|Great prices, pro...|2017-06-22T13:24:...|10487654216930859...|
|0x89de09b4a848543...|     4|Usually crowded w...|2017-06-19T23:38:...|10310030037917646...|
|0x89de09b4a848543...|     1|pharmacy slow as ...|2017-06-16T15:04:...|11320908474528403...|
|0x89de09b4a848543...|     5|             Love it|2017-12-25T01:15:...|11173201801956000...|
|0x89de09b4a848543...|     5|         Best deals!|2018-03-01T14:13:...|10166825774143799...|
|0x89de09b4a848543...|     3|Its a pretty norm...|2018-01-20T15:39:...|11370958489473963...|
|0x89de09b4a848543...|     4|Walmart, yo. They...|2017-06-06T11:23:...

In [ ]:
df_estados = df_estados.drop('time')

In [ ]:
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import col, when

In [ ]:
from pyspark.sql.functions import length

In [ ]:
# Mostrar el DataFrame final
df_estados.show()

+--------------------+------+--------------------+--------------------+
|             gmap_id|rating|                text|             user_id|
+--------------------+------+--------------------+--------------------+
|0x89de09b4a848543...|     3|Seems to be going...|11294936078038757...|
|0x89de09b4a848543...|     4|Great prices, pro...|10487654216930859...|
|0x89de09b4a848543...|     4|Usually crowded w...|10310030037917646...|
|0x89de09b4a848543...|     1|pharmacy slow as ...|11320908474528403...|
|0x89de09b4a848543...|     5|             Love it|11173201801956000...|
|0x89de09b4a848543...|     5|         Best deals!|10166825774143799...|
|0x89de09b4a848543...|     3|Its a pretty norm...|11370958489473963...|
|0x89de09b4a848543...|     4|Walmart, yo. They...|11812869879284333...|
|0x89de09b4a848543...|     4|Well stocked and ...|10594921432141909...|
|0x89de09b4a848543...|     4|          Nice place|11345934252183103...|
|0x89de09b4a848543...|     4|Your average Walmart|10187075909416

In [ ]:
from pyspark.sql.functions import col, regexp_replace

In [ ]:
# Reemplazar las comas en la columna 'text'
df_estados = df_estados.withColumn('text', regexp_replace(col('text'), ',', ''))

# Mostrar el DataFrame final
df_estados.show()

+--------------------+------+--------------------+--------------------+
|             gmap_id|rating|                text|             user_id|
+--------------------+------+--------------------+--------------------+
|0x89de09b4a848543...|     3|Seems to be going...|11294936078038757...|
|0x89de09b4a848543...|     4|Great prices prod...|10487654216930859...|
|0x89de09b4a848543...|     4|Usually crowded w...|10310030037917646...|
|0x89de09b4a848543...|     1|pharmacy slow as ...|11320908474528403...|
|0x89de09b4a848543...|     5|             Love it|11173201801956000...|
|0x89de09b4a848543...|     5|         Best deals!|10166825774143799...|
|0x89de09b4a848543...|     3|Its a pretty norm...|11370958489473963...|
|0x89de09b4a848543...|     4|Walmart yo. They ...|11812869879284333...|
|0x89de09b4a848543...|     4|Well stocked and ...|10594921432141909...|
|0x89de09b4a848543...|     4|          Nice place|11345934252183103...|
|0x89de09b4a848543...|     4|Your average Walmart|10187075909416

In [ ]:
df_estados.count()

5562771

In [ ]:
from pyspark.sql.functions import col

# Verificar si la columna 'rating' contiene solo valores numéricos
has_only_numeric_values = df_estados.filter(col("user_id").cast("float").isNotNull()).count() == df_estados.count()

if has_only_numeric_values:
    print("La columna 'user_id' contiene solo valores numéricos.")
else:
    print("La columna 'user_id' contiene valores que no son numéricos.")


La columna 'user_id' contiene valores que no son numéricos.


In [ ]:
# Verificar si la columna 'rating' contiene solo valores numéricos
has_only_numeric_values = df_estados.filter(col("rating").cast("float").isNotNull()).count() == df_estados.count()

if has_only_numeric_values:
    print("La columna 'user_id' contiene solo valores numéricos.")
else:
    print("La columna 'user_id' contiene valores que no son numéricos.")

La columna 'user_id' contiene valores que no son numéricos.


In [ ]:
# Calcular la longitud de cada valor en la columna
lengths_df = df_estados.withColumn("length", length("gmap_id"))

# Verificar si todos los valores tienen la misma longitud
same_length = lengths_df.select(countDistinct("length")).collect()[0][0]

if same_length == 1:
    print("La columna 'gmap_id' tiene la misma longitud en todas las filas.")
else:
    print("La columna 'gmap_id' tiene diferentes longitudes en las filas.")

La columna 'gmap_id' tiene diferentes longitudes en las filas.


In [ ]:
# Convertir la columna 'rating' a tipo float, reemplazando los valores no numéricos por 0
df_estados = df_estados.withColumn("rating", when(col("rating").cast("float").isNotNull(), col("rating").cast("float")).otherwise(0.0))

# Verificar si la columna 'rating' contiene solo valores numéricos
has_only_numeric_values = df_estados.filter(col("rating").cast("float").isNotNull()).count() == df_estados.count()

if has_only_numeric_values:
    print("La columna 'rating' ahora contiene solo valores numéricos.")
else:
    print("La columna 'rating' contiene valores que no son numéricos.")

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Cuenta el número de nulos en cada columna

def conteo_nulos(dataframe):
  conteo_nulos_por_columna = dataframe.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in dataframe.columns])

  # Muestra el resultado
  conteo_nulos_por_columna.show()

In [ ]:
conteo_nulos(df_estados)

+-------+------+-------+-------+
|gmap_id|rating|   text|user_id|
+-------+------+-------+-------+
|     96|     0|2408825| 855606|
+-------+------+-------+-------+



In [ ]:
# Eliminar filas donde 'gmap_id' está vacío
df_estados_sin_nulos = df_estados.dropna(subset=['gmap_id'])

# Verificar si se eliminaron las filas nulas
conteo_nulos(df_estados_sin_nulos)

+-------+------+-------+-------+
|gmap_id|rating|   text|user_id|
+-------+------+-------+-------+
|      0|     0|2408778| 855525|
+-------+------+-------+-------+



In [ ]:
# Eliminar filas donde 'gmap_id' está vacío
df_estados_sin_nulos = df_estados_sin_nulos.dropna(subset=['user_id'])

# Verificar si se eliminaron las filas nulas
conteo_nulos(df_estados_sin_nulos)

+-------+------+-------+-------+
|gmap_id|rating|   text|user_id|
+-------+------+-------+-------+
|      0|     0|2121646|      0|
+-------+------+-------+-------+



In [ ]:
# Eliminar duplicados basándote en todas las columnas
df_no_duplicates = df_estados_sin_nulos.dropDuplicates()

# Contar las filas después de eliminar duplicados
print("Número de filas después de eliminar duplicados:", df_no_duplicates.count())

Número de filas después de eliminar duplicados: 4462681


In [ ]:
df_repartitioned = df_estados.repartition(20)  # 20 particiones para obtener 20 archivos CSV

In [ ]:
from pyspark.sql.functions import col, regexp_replace

# Reemplazar las comas en todas las columnas
df_no_commas = df_no_duplicates.select([
    regexp_replace(col(column), ',', '').alias(column)
    for column in df_no_duplicates.columns
])

# Mostrar el DataFrame final sin comas en ninguna columna
df_no_commas.show()

+--------------------+------+--------------------+--------------------+
|             gmap_id|rating|                text|             user_id|
+--------------------+------+--------------------+--------------------+
|0x89de09b4a848543...|   3.0|Good prices  diff...|10223400974999296...|
|0x89c2590740f807b...|   1.0|The spa itself is...|10136332133563669...|
|0x89dcdf75cd22fec...|   5.0|Ended up making a...|10591692115098954...|
|0x89dcdf75cd22fec...|   5.0|Great hiking plac...|10487774266149854...|
|0x89dcdf75cd22fec...|   4.0|A beautiful area ...|10395750245222615...|
|0x89dcdf75cd22fec...|   5.0|great for hiking ...|10122918718989754...|
|0x89dcdf75cd22fec...|   5.0|One of the best s...|10899093470869347...|
|0x89c25c63b9c4659...|   1.0|I'm giving one st...|10033669893008216...|
|0x4ccabd49ce205bd...|   4.0|Great food busy a...|11754469105734698...|
|0x89c2632c75b49d8...|   5.0|Good rolled ice c...|11738861751292525...|
|0x89c266f78858ad2...|   5.0|Beautiful lounge....|11551455173490

In [ ]:
# Reemplazar las comillas en todas las columnas
df_no_quote = df_no_commas.select([
    regexp_replace(col(column), '"', '').alias(column)
    for column in df_no_duplicates.columns
])

In [ ]:
1

In [ ]:
# Eliminar los daños mayores a 5 o menores a 0 de la columna "rating"
df_with_numeric_rating = df_with_numeric.filter((col("rating") >= 0) & (col("rating") <= 5))

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from textblob import TextBlob

# Función para determinar el sentimiento basado en el texto y la calificación
def get_sentiment(text, rating):
    if text:
        analysis = TextBlob(text)
        if analysis.sentiment.polarity > 0:
            return 1  # Sentimiento positivo
        elif analysis.sentiment.polarity < 0:
            return 2  # Sentimiento negativo
        else:
            return 0  # Sentimiento neutro
    else:
        if rating is not None:
            if rating >= 4:
                return 1  # Calificación alta considerada como sentimiento positivo
            elif rating <= 2:
                return 2  # Calificación baja considerada como sentimiento negativo
            else:
                return 0  # Calificación intermedia considerada como sentimiento neutro
        else:
            return 0  # Si tanto el texto como la calificación están vacíos, asignar 0

# Registrar la función como UDF
sentiment_udf = udf(lambda text, rating: get_sentiment(text, rating), IntegerType())



In [ ]:
# Aplicar la función a la columna "review" y crear una nueva columna "sentiment"
df_with_rating_sentiment = df_with_numeric_rating.withColumn("sentiment", sentiment_udf("text", "rating"))

In [ ]:
df_with_rating_sentiment.printSchema()

root
 |-- gmap_id: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- sentiment: integer (nullable = true)



In [ ]:
df_with_rating_sentiment.show()

+--------------------+------+--------------------+--------------------+--------------+----------------+
|             gmap_id|rating|                text|             user_id|text_sentiment|rating_sentiment|
+--------------------+------+--------------------+--------------------+--------------+----------------+
|0x89de09b4a848543...|   3.0|Good prices  diff...|10223400974999296...|             1|               0|
|0x89c2590740f807b...|   1.0|The spa itself is...|10136332133563669...|             1|               2|
|0x89dcdf75cd22fec...|   5.0|Ended up making a...|10591692115098954...|             1|               1|
|0x89dcdf75cd22fec...|   5.0|Great hiking plac...|10487774266149854...|             1|               1|
|0x89dcdf75cd22fec...|   4.0|A beautiful area ...|10395750245222615...|             1|               1|
|0x89dcdf75cd22fec...|   5.0|great for hiking ...|10122918718989754...|             1|               1|
|0x89dcdf75cd22fec...|   5.0|One of the best s...|10899093470869

In [ ]:
# Ruta al archivo CSV local
file_path = '/content/drive/MyDrive/Colab-Notebooks/Datos-Limpios/estados-limpioswsen9.csv'

# Escribe el DataFrame a un solo archivo CSV localmente con separador ","
df_with_rating_sentiment.coalesce(4).write.csv(file_path, header=True, sep=',')

# **YELP**

# **BUSSINES**

In [ ]:
ruta_archivo = '/content/drive/MyDrive/Colab-Notebooks/Dataset-Limpios/business-limpio.csv'

df_buss = pd.read_csv(ruta_archivo)

In [ ]:
df_buss.head(10)

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",NaN
1,1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
5,5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
6,6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,PA,63144,38.627695,-90.340465,2.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
7,7,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,PA,33707,27.766590,-82.732983,3.5,5,1,NaN,"Synagogues, Religious Organizations","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
8,8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,TN,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...","Pubs, Restaurants, Italian, Bars, American (Tr...",NaN
9,9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,MO,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."


In [ ]:
df_buss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    150346 non-null  int64  
 1   business_id   150346 non-null  object 
 2   name          150346 non-null  object 
 3   address       145219 non-null  object 
 4   city          150346 non-null  object 
 5   state         150346 non-null  object 
 6   postal_code   150273 non-null  object 
 7   latitude      150346 non-null  float64
 8   longitude     150346 non-null  float64
 9   stars         150346 non-null  float64
 10  review_count  150346 non-null  int64  
 11  is_open       150346 non-null  int64  
 12  attributes    136602 non-null  object 
 13  categories    150243 non-null  object 
 14  hours         127123 non-null  object 
dtypes: float64(3), int64(3), object(9)
memory usage: 17.2+ MB


In [ ]:
df_buss.count()

Unnamed: 0      150346
business_id     150346
name            150346
address         145219
city            150346
state           150346
postal_code     150273
latitude        150346
longitude       150346
stars           150346
review_count    150346
is_open         150346
attributes      136602
categories      150243
hours           127123
dtype: int64

In [ ]:
# Eliminar filas con valores NaN en la columna 'categories'
df_buss = df_buss.dropna(subset=['categories'])

In [ ]:
# Filtrar las filas donde la columna 'categories' contiene la palabra 'Restaurants'
restaurants_df = df_buss[df_buss['categories'].str.contains('Restaurants')]

In [ ]:
df_buss.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,PA,63144,38.627695,-90.340465,2.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
7,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,PA,33707,27.766590,-82.732983,3.5,5,1,NaN,"Synagogues, Religious Organizations","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,TN,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...","Pubs, Restaurants, Italian, Bars, American (Tr...",NaN
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,MO,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."


In [ ]:
if 'Unnamed: 0' in df_buss.columns:
    df_buss = df_buss.drop(columns=['Unnamed: 0'])


In [ ]:
# Eliminar las columnas 'attributes' y 'hours'.. codigo postal tambien
df_buss = df_buss.drop(columns=['postal_code'])

In [ ]:
# Obtener la cantidad de decimales en la columna de latitud
df_buss['lat_decimal_count'] = df_buss['latitude'].apply(lambda x: len(str(x).split('.')[-1]))

# Obtener la cantidad de decimales en la columna de longitud
df_buss['lon_decimal_count'] = df_buss['longitude'].apply(lambda x: len(str(x).split('.')[-1]))

In [ ]:
# Verificar si todas las filas tienen la misma cantidad de decimales en latitud
lat_decimal_same = df_buss['lat_decimal_count'].nunique() == 1

# Verificar si todas las filas tienen la misma cantidad de decimales en longitud
lon_decimal_same = df_buss['lon_decimal_count'].nunique() == 1

In [ ]:

# Mostrar el resultado
print("Latitud tiene la misma cantidad de decimales en todas las filas:", lat_decimal_same)
print("Longitud tiene la misma cantidad de decimales en todas las filas:", lon_decimal_same)

# Eliminar las columnas auxiliares
df_buss.drop(['lat_decimal_count', 'lon_decimal_count'], axis=1, inplace=True)

Latitud tiene la misma cantidad de decimales en todas las filas: True
Longitud tiene la misma cantidad de decimales en todas las filas: True


In [ ]:
# Función para completar con ceros a la derecha hasta llegar a la cantidad deseada de decimales
def completar_decimales(valor, decimales):
    partes = str(valor).split('.')
    if len(partes) == 1:
        return f"{valor}.{'0' * decimales}"
    elif len(partes[1]) < decimales:
        return f"{valor}{'0' * (decimales - len(partes[1]))}"
    else:
        return valor

# Número de decimales deseado
decimales = 6

# Redondear latitud y longitud a la cantidad deseada de decimales
df_buss['latitude'] = df_buss['latitude'].round(decimales)
df_buss['longitude'] = df_buss['longitude'].round(decimales)

# Completar con ceros a la derecha si tienen menos de 6 decimales
df_buss['latitude'] = df_buss['latitude'].apply(lambda x: completar_decimales(x, decimales))
df_buss['longitude'] = df_buss['longitude'].apply(lambda x: completar_decimales(x, decimales))

# Verificar si todas las filas tienen la misma cantidad de decimales
latitud_decimales_iguales = df_buss['latitude'].apply(lambda x: len(str(x).split('.')[-1])) == decimales
longitud_decimales_iguales = df_buss['longitude'].apply(lambda x: len(str(x).split('.')[-1])) == decimales

print("Latitud tiene la misma cantidad de decimales en todas las filas:", latitud_decimales_iguales.all())
print("Longitud tiene la misma cantidad de decimales en todas las filas:", longitud_decimales_iguales.all())


Latitud tiene la misma cantidad de decimales en todas las filas: True
Longitud tiene la misma cantidad de decimales en todas las filas: True


In [ ]:
# Eliminar filas con valores NaN en la columna 'categories'
df_buss = df_buss.dropna(subset=['categories'])

# Filtrar las filas donde la columna 'categories' contiene la palabra 'Restaurants' y el estado es NY o CA
restaurants_df = df_buss[(df_buss['categories'].str.contains('Restaurants')) & (df_buss['state'].isin(['NY', 'CA']))]


In [ ]:
restaurants_df.head(10)

,business_id,name,address,city,state,latitude,longitude,stars,review_count,is_open,categories
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
29,sqSqqLy0sN8n2IZrAbzidQ,Domino's Pizza,3001 Highway 31 W,White House,CA,36.464747,-86.659187,3.5,8,1,"Pizza, Chicken Wings, Sandwiches, Restaurants"
88,LhpPSrulqVeTyJeK2xydvQ,Fresh Fruits & Salads,114 N 3rd St,Camden,CA,39.946690,-75.123327,4.5,6,1,"Juice Bars & Smoothies, Restaurants, Fruits & ..."
222,8c0r7olQSYGcws0bTd3ikw,Zesty Tsunami,6614 Osteen Rd,New Port Richey,CA,28.250218,-82.682441,4.0,5,0,"Hawaiian, Food, Food Trucks, Street Vendors, R..."
409,QWqKTWQ2OiDgo3dzNkpung,Jong Ka Jib,6600 N 5th St,Philadelphia,CA,40.052064,-75.127394,4.5,351,1,"Korean, Restaurants"
618,3uvFklejzRPwEPEZQ5vHYQ,Wawa,100 E Old Baltimore Pike,Media,CA,39.916176,-75.387655,3.5,14,1,"Delis, Convenience Stores, Restaurants, Food, ..."
694,jsQQu9rVerR3OGxIa0hRQQ,Kelly's Taproom,1107 Lancaster Ave,Bryn Mawr,CA,40.024588,-75.324239,3.0,63,1,"Bars, Beer Bar, American (Traditional), Nightl..."
821,JomDoQafikclSVVCPkWjnA,Cedar Park Cafe,4914 Baltimore Ave,Philadelphia,CA,39.947960,-75.221980,4.0,301,1,"Diners, Restaurants, Sandwiches, Breakfast & B..."
850,wMyHVb54Ne9SNasa4kjOCg,Carnivore,5257 Shaw Ave,St. Louis,CA,38.618412,-90.274182,4.5,281,1,"Restaurants, Breakfast & Brunch, Steakhouses, ..."
875,7Ct4QuWaY63DkAOXqEiVGA,Hong Kong Chinese Restaurant,326 N Chester Pike,Glenolden,CA,39.903317,-75.284408,3.0,18,1,"Chinese, Restaurants"


In [ ]:
restaurants_df.count()

business_id     1812
name            1812
address         1799
city            1812
state           1812
latitude        1812
longitude       1812
stars           1812
review_count    1812
is_open         1812
categories      1812
dtype: int64

In [ ]:
# Ruta al archivo CSV
file_path = '/content/drive/MyDrive/Colab-Notebooks/Datos-Limpios/bussines-limpio.csv'

# Guardar el DataFrame en un archivo CSV en la carpeta file_path
restaurants_df.to_csv(file_path, sep=';', index=False)


# **checkin**

In [ ]:
ruta_archivo = '/content/drive/MyDrive/Colab-Notebooks/Dataset-Limpios/checkin-limpio.csv'
df_ck = pd.read_csv(ruta_archivo)

In [ ]:
df_ck.head(10)

,Unnamed: 0,checkin_id,business_id,date
0,0,0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13 21:10:56
1,1,0,---kPU91CF4Lq2-WlRu9Lw,2020-06-02 22:18:06
2,2,0,---kPU91CF4Lq2-WlRu9Lw,2020-07-24 22:42:27
3,3,0,---kPU91CF4Lq2-WlRu9Lw,2020-10-24 21:36:13
4,4,0,---kPU91CF4Lq2-WlRu9Lw,2020-12-09 21:23:33
5,5,0,---kPU91CF4Lq2-WlRu9Lw,2021-01-20 17:34:57
6,6,0,---kPU91CF4Lq2-WlRu9Lw,2021-04-30 21:02:03
7,7,0,---kPU91CF4Lq2-WlRu9Lw,2021-05-25 21:16:54
8,8,0,---kPU91CF4Lq2-WlRu9Lw,2021-08-06 21:08:08
9,9,0,---kPU91CF4Lq2-WlRu9Lw,2021-10-02 15:15:42


In [ ]:
# Eliminar las primeras 2 columnas
df_ck_cleaned = df_ck.drop(columns=df_ck.columns[:2])

# Mostrar las primeras filas del DataFrame limpio
df_ck_cleaned.head(10)


,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13 21:10:56
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02 22:18:06
2,---kPU91CF4Lq2-WlRu9Lw,2020-07-24 22:42:27
3,---kPU91CF4Lq2-WlRu9Lw,2020-10-24 21:36:13
4,---kPU91CF4Lq2-WlRu9Lw,2020-12-09 21:23:33
5,---kPU91CF4Lq2-WlRu9Lw,2021-01-20 17:34:57
6,---kPU91CF4Lq2-WlRu9Lw,2021-04-30 21:02:03
7,---kPU91CF4Lq2-WlRu9Lw,2021-05-25 21:16:54
8,---kPU91CF4Lq2-WlRu9Lw,2021-08-06 21:08:08
9,---kPU91CF4Lq2-WlRu9Lw,2021-10-02 15:15:42


In [ ]:
# Convertir la columna 'date' a tipo datetime
df_ck_cleaned['date'] = pd.to_datetime(df_ck_cleaned['date'])

# Extraer solo la parte de la fecha (año-mes-día)
df_ck_cleaned['date'] = df_ck_cleaned['date'].dt.date

# Mostrar las primeras filas del DataFrame con la fecha formateada
df_ck_cleaned.head(10)


,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02
2,---kPU91CF4Lq2-WlRu9Lw,2020-07-24
3,---kPU91CF4Lq2-WlRu9Lw,2020-10-24
4,---kPU91CF4Lq2-WlRu9Lw,2020-12-09
5,---kPU91CF4Lq2-WlRu9Lw,2021-01-20
6,---kPU91CF4Lq2-WlRu9Lw,2021-04-30
7,---kPU91CF4Lq2-WlRu9Lw,2021-05-25
8,---kPU91CF4Lq2-WlRu9Lw,2021-08-06
9,---kPU91CF4Lq2-WlRu9Lw,2021-10-02


In [ ]:
df_ck_cleaned.count()

business_id    13356875
date           13356875
dtype: int64

In [ ]:
# Ruta al archivo CSV
file_path = '/content/drive/MyDrive/Colab-Notebooks/Datos-Limpios/checkin-limpio.csv'

# Guardar el DataFrame en un archivo CSV en la carpeta file_path
df_ck_cleaned.to_csv(file_path, sep=';', index=False)

# **tip**

In [ ]:
ruta_archivo = '/content/drive/MyDrive/Colab-Notebooks/Dataset-Limpios/tip-limpio.csv'
df_tip = pd.read_csv(ruta_archivo)

In [ ]:
df_tip.head(10)

,Unnamed: 0,tip_id,user_id,business_id,tip,date,compliment_count
0,0,1,rCumu_NyXfbyq16cP8vOEw,cXSyVvOr9YRN9diDkaWs0Q,Simply the best breakfast around!! The staff i...,2009-04-16 13:11:49,0
1,1,2,LT_JU6bY75H918eKnnXeVQ,kLYhipAEvdT1ORfluX5gbw,Crunchy French toast is amazing!,2009-04-16 17:15:29,0
2,2,3,sxZX1armKzagQxDnbvPnkw,gTC8IQ_i8zXytWSly3Ttvg,"Lunch specials are great, especially their hom...",2009-04-16 19:59:35,0
3,3,4,KI4UutWtai0UKZ6ZK2pg2Q,P3bw4h4kCaMaYqGGNLt3Jg,"Low selection, quiet",2009-04-16 23:45:46,0
4,4,5,C14KW1jjKM8QlYfMyzP1sQ,aUjJ_x1KuvWmqIErhdaHzg,"Pretty good pad thai, but when I asked for lim...",2009-04-17 00:34:55,0
5,5,6,8IcTnw2hmu5xjElM1uZigw,DU4y4jJPE96-hBN_jLbCbg,Tries to cater for special diets. Let them kno...,2009-04-17 02:03:49,0
6,6,7,osLu8P3SkCJJQoMpx_kc3w,5k0-j1Nuoj0CrYzuhNqWxg,The breaded palomilla steak is excellent. Huge...,2009-04-17 02:20:20,0
7,7,8,aD-rsDe2S0e7jTkYD6ULaQ,iSRTaT9WngzB8JJ2YKJUig,"While waiting in line, have one person go get ...",2009-04-17 18:14:34,0
8,8,9,dYUP-Nw3mHPnO8_48DXExQ,3GjYa0zpRfNXScHguQdDDg,Busy at lunch. Call ahead.,2009-04-17 22:00:26,0
9,9,10,KI4UutWtai0UKZ6ZK2pg2Q,DKXeB-JIou5XPI49CzITuQ,Hot and sour soup. Best ever.,2009-04-18 01:02:56,0


In [ ]:
# Eliminar las primeras 2 columnas
df_tip = df_tip.drop(columns=df_tip.columns[:2])

# Mostrar las primeras filas del DataFrame limpio
df_tip.head(10)


,user_id,business_id,tip,date,compliment_count
0,rCumu_NyXfbyq16cP8vOEw,cXSyVvOr9YRN9diDkaWs0Q,Simply the best breakfast around!! The staff i...,2009-04-16 13:11:49,0
1,LT_JU6bY75H918eKnnXeVQ,kLYhipAEvdT1ORfluX5gbw,Crunchy French toast is amazing!,2009-04-16 17:15:29,0
2,sxZX1armKzagQxDnbvPnkw,gTC8IQ_i8zXytWSly3Ttvg,"Lunch specials are great, especially their hom...",2009-04-16 19:59:35,0
3,KI4UutWtai0UKZ6ZK2pg2Q,P3bw4h4kCaMaYqGGNLt3Jg,"Low selection, quiet",2009-04-16 23:45:46,0
4,C14KW1jjKM8QlYfMyzP1sQ,aUjJ_x1KuvWmqIErhdaHzg,"Pretty good pad thai, but when I asked for lim...",2009-04-17 00:34:55,0
5,8IcTnw2hmu5xjElM1uZigw,DU4y4jJPE96-hBN_jLbCbg,Tries to cater for special diets. Let them kno...,2009-04-17 02:03:49,0
6,osLu8P3SkCJJQoMpx_kc3w,5k0-j1Nuoj0CrYzuhNqWxg,The breaded palomilla steak is excellent. Huge...,2009-04-17 02:20:20,0
7,aD-rsDe2S0e7jTkYD6ULaQ,iSRTaT9WngzB8JJ2YKJUig,"While waiting in line, have one person go get ...",2009-04-17 18:14:34,0
8,dYUP-Nw3mHPnO8_48DXExQ,3GjYa0zpRfNXScHguQdDDg,Busy at lunch. Call ahead.,2009-04-17 22:00:26,0
9,KI4UutWtai0UKZ6ZK2pg2Q,DKXeB-JIou5XPI49CzITuQ,Hot and sour soup. Best ever.,2009-04-18 01:02:56,0


In [ ]:
df_tip.head(10)

,user_id,business_id,tip,date
0,rCumu_NyXfbyq16cP8vOEw,cXSyVvOr9YRN9diDkaWs0Q,Simply the best breakfast around!! The staff i...,2009-04-16 13:11:49
1,LT_JU6bY75H918eKnnXeVQ,kLYhipAEvdT1ORfluX5gbw,Crunchy French toast is amazing!,2009-04-16 17:15:29
2,sxZX1armKzagQxDnbvPnkw,gTC8IQ_i8zXytWSly3Ttvg,"Lunch specials are great, especially their hom...",2009-04-16 19:59:35
3,KI4UutWtai0UKZ6ZK2pg2Q,P3bw4h4kCaMaYqGGNLt3Jg,"Low selection, quiet",2009-04-16 23:45:46
4,C14KW1jjKM8QlYfMyzP1sQ,aUjJ_x1KuvWmqIErhdaHzg,"Pretty good pad thai, but when I asked for lim...",2009-04-17 00:34:55
5,8IcTnw2hmu5xjElM1uZigw,DU4y4jJPE96-hBN_jLbCbg,Tries to cater for special diets. Let them kno...,2009-04-17 02:03:49
6,osLu8P3SkCJJQoMpx_kc3w,5k0-j1Nuoj0CrYzuhNqWxg,The breaded palomilla steak is excellent. Huge...,2009-04-17 02:20:20
7,aD-rsDe2S0e7jTkYD6ULaQ,iSRTaT9WngzB8JJ2YKJUig,"While waiting in line, have one person go get ...",2009-04-17 18:14:34
8,dYUP-Nw3mHPnO8_48DXExQ,3GjYa0zpRfNXScHguQdDDg,Busy at lunch. Call ahead.,2009-04-17 22:00:26
9,KI4UutWtai0UKZ6ZK2pg2Q,DKXeB-JIou5XPI49CzITuQ,Hot and sour soup. Best ever.,2009-04-18 01:02:56


In [ ]:
# Convertir la columna 'date' a tipo datetime
df_tip['date'] = pd.to_datetime(df_tip['date'])

# Extraer solo la parte de la fecha (año-mes-día)
df_tip['date'] = df_tip['date'].dt.date

# Mostrar las primeras filas del DataFrame con la fecha formateada
df_tip.head(10)

,user_id,business_id,tip,date
0,rCumu_NyXfbyq16cP8vOEw,cXSyVvOr9YRN9diDkaWs0Q,Simply the best breakfast around!! The staff i...,2009-04-16
1,LT_JU6bY75H918eKnnXeVQ,kLYhipAEvdT1ORfluX5gbw,Crunchy French toast is amazing!,2009-04-16
2,sxZX1armKzagQxDnbvPnkw,gTC8IQ_i8zXytWSly3Ttvg,"Lunch specials are great, especially their hom...",2009-04-16
3,KI4UutWtai0UKZ6ZK2pg2Q,P3bw4h4kCaMaYqGGNLt3Jg,"Low selection, quiet",2009-04-16
4,C14KW1jjKM8QlYfMyzP1sQ,aUjJ_x1KuvWmqIErhdaHzg,"Pretty good pad thai, but when I asked for lim...",2009-04-17
5,8IcTnw2hmu5xjElM1uZigw,DU4y4jJPE96-hBN_jLbCbg,Tries to cater for special diets. Let them kno...,2009-04-17
6,osLu8P3SkCJJQoMpx_kc3w,5k0-j1Nuoj0CrYzuhNqWxg,The breaded palomilla steak is excellent. Huge...,2009-04-17
7,aD-rsDe2S0e7jTkYD6ULaQ,iSRTaT9WngzB8JJ2YKJUig,"While waiting in line, have one person go get ...",2009-04-17
8,dYUP-Nw3mHPnO8_48DXExQ,3GjYa0zpRfNXScHguQdDDg,Busy at lunch. Call ahead.,2009-04-17
9,KI4UutWtai0UKZ6ZK2pg2Q,DKXeB-JIou5XPI49CzITuQ,Hot and sour soup. Best ever.,2009-04-18


In [ ]:
# Ruta al archivo CSV
file_path = '/content/drive/MyDrive/Colab-Notebooks/Datos-Limpios/tip-limpio.csv'

# Guardar el DataFrame en un archivo CSV en la carpeta file_path
df_tip.to_csv(file_path, sep=';', index=False)

# **user**

In [ ]:
ruta_archivo = '/content/drive/MyDrive/Colab-Notebooks/Dataset-Limpios/user-limpio.csv'
df_us = pd.read_csv(ruta_archivo)

In [ ]:
df_us.head(10)

,Unnamed: 0,user_id,name,review_count,yelping_since,useful,funny,cool,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,0,fFGPBtsutYpn3A155Sf75Q,Brandon,194,2004-10-12 08:46:11,250,103,121,245,26,...,1,3,11,0,6,11,21,21,1,3
1,1,nkN_do3fJ9xekchVC-v68A,Jeremy,1366,2004-10-12 08:46:43,18524,10049,15141,6849,2107,...,178,152,249,45,890,1254,2150,2150,473,581
2,2,wqoXYLWmpkEH0YvTmHBsJQ,Michael,398,2004-10-12 08:51:07,1393,734,662,2454,156,...,22,13,24,4,90,98,168,168,27,33
3,3,co_jK_x-CvK2Z3ZrJLz1SQ,j,12,2004-10-12 09:36:53,21,3,4,7,0,...,0,0,0,0,1,0,0,0,0,0
4,4,23J4vG9_xxxdnmi8CBX7Ng,Joan,1674,2004-10-12 12:29:35,21509,15514,19972,3926,1425,...,292,290,342,98,1480,4961,5126,5126,1904,3291
5,5,LFLpi-FfgDW4zpcdz3hfSQ,Elena,37,2004-10-12 13:27:53,31,16,30,21,8,...,3,2,3,0,4,2,4,4,5,4
6,6,xIYFwwrwxSFyz1IjVfHTkg,Lindsay,66,2004-10-12 14:10:17,153,19,24,108,4,...,4,0,0,0,5,11,0,0,0,0
7,7,KKOM5feU4TzZmUEY24y02Q,Eric,212,2004-10-12 14:42:04,288,94,102,50,27,...,3,2,0,8,9,9,4,4,4,0
8,8,y8lz_6KWGOQ9jcTyIdwTLQ,Patrick,66,2004-10-12 15:25:43,87,25,44,108,2,...,1,0,1,0,3,3,7,7,2,0
9,9,XG-JynrbKiiNaqT4ffdI_w,Ossama,117,2004-10-12 20:07:51,61,18,29,61,6,...,3,1,2,0,2,0,4,4,2,0


In [ ]:
# Conteo de valores nulos en cada columna
conteo_nulos = df_us.isnull().sum()

# Mostrar el conteo de valores nulos
print(conteo_nulos)


Unnamed: 0            0
user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64


In [ ]:
# Eliminar filas con valores nulos en la columna 'columna'
df_us = df_us.dropna(subset=['name'])


In [ ]:
# Eliminar la primera columna sin nombre
df_us = df_us.iloc[:, 1:]


In [ ]:
df_us.head(10)

,user_id,name,review_count,useful,funny,cool,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,fFGPBtsutYpn3A155Sf75Q,Brandon,194,250,103,121,245,26,4.18,7,1,3,11,0,6,11,21,21,1,3
1,nkN_do3fJ9xekchVC-v68A,Jeremy,1366,18524,10049,15141,6849,2107,3.70,1454,178,152,249,45,890,1254,2150,2150,473,581
2,wqoXYLWmpkEH0YvTmHBsJQ,Michael,398,1393,734,662,2454,156,3.61,111,22,13,24,4,90,98,168,168,27,33
3,co_jK_x-CvK2Z3ZrJLz1SQ,j,12,21,3,4,7,0,2.50,0,0,0,0,0,1,0,0,0,0,0
4,23J4vG9_xxxdnmi8CBX7Ng,Joan,1674,21509,15514,19972,3926,1425,4.37,3664,292,290,342,98,1480,4961,5126,5126,1904,3291
5,LFLpi-FfgDW4zpcdz3hfSQ,Elena,37,31,16,30,21,8,3.92,4,3,2,3,0,4,2,4,4,5,4
6,xIYFwwrwxSFyz1IjVfHTkg,Lindsay,66,153,19,24,108,4,3.91,0,4,0,0,0,5,11,0,0,0,0
7,KKOM5feU4TzZmUEY24y02Q,Eric,212,288,94,102,50,27,3.73,3,3,2,0,8,9,9,4,4,4,0
8,y8lz_6KWGOQ9jcTyIdwTLQ,Patrick,66,87,25,44,108,2,4.00,1,1,0,1,0,3,3,7,7,2,0
9,XG-JynrbKiiNaqT4ffdI_w,Ossama,117,61,18,29,61,6,3.76,2,3,1,2,0,2,0,4,4,2,0


In [ ]:
# Eliminar la columna 'yelping_since'
df_us = df_us.drop(columns=['yelping_since'])


In [ ]:
# Lista de columnas a verificar
columnas_numericas = ['review_count', 'useful', 'funny', 'cool', 'friends', 'fans',
                      'compliment_hot', 'compliment_more', 'compliment_profile',
                      'compliment_cute', 'compliment_list', 'compliment_note',
                      'compliment_plain', 'compliment_cool', 'compliment_funny',
                      'compliment_writer', 'compliment_photos']

# Verificar si las columnas contienen solo valores numéricos
resultados = df_us[columnas_numericas].apply(lambda x: pd.to_numeric(x, errors='coerce').notnull().all())

# Mostrar los resultados
print(resultados)


review_count          True
useful                True
funny                 True
cool                  True
friends               True
fans                  True
compliment_hot        True
compliment_more       True
compliment_profile    True
compliment_cute       True
compliment_list       True
compliment_note       True
compliment_plain      True
compliment_cool       True
compliment_funny      True
compliment_writer     True
compliment_photos     True
dtype: bool


In [ ]:
# Intentar convertir la columna 'average_stars' a números
df_us['average_stars_numeric'] = pd.to_numeric(df_us['average_stars'], errors='coerce')

# Verificar si todos los valores se convirtieron correctamente
todos_numeros = df_us['average_stars_numeric'].notnull().all()

# Mostrar el resultado
print("La columna 'average_stars' contiene solo valores numéricos:", todos_numeros)

# Eliminar la columna temporal
df_us.drop('average_stars_numeric', axis=1, inplace=True)

La columna 'average_stars' contiene solo valores numéricos: True


In [ ]:
# Verificar si todos los valores de 'name' contienen solo letras
solo_letras = df_us['name'].str.isalpha().all()

# Mostrar el resultado
print("La columna 'name' contiene solo letras:", solo_letras)

La columna 'name' contiene solo letras: False


In [ ]:
# Convertir todos los valores de 'name' a minúsculas y eliminar los caracteres que no son letras
df_us['name'] = df_us['name'].str.replace(r'[^a-zA-Z\s]', '').str.strip()

# Verificar nuevamente si todos los valores de 'name' contienen solo letras
solo_letras = df_us['name'].str.isalpha().all()

# Mostrar el resultado
print("La columna 'name' contiene solo letras después de la limpieza:", solo_letras)


La columna 'name' contiene solo letras después de la limpieza: False


In [ ]:
# Ruta al archivo CSV
file_path = '/content/drive/MyDrive/Colab-Notebooks/Datos-Limpios/user-limpio2.csv'

# Guardar el DataFrame en un archivo CSV en la carpeta file_path
df_us.to_csv(file_path, sep=';', index=False)

# **REVIEWS - YELP**

In [ ]:
# Ruta a la carpeta que contiene los archivos CSV en Google Drive
carpeta = '/content/drive/MyDrive/Colab-Notebooks/Dataset-Limpios/review-limpio.csv'

# Listar archivos en la carpeta
archivos = os.listdir(carpeta)

# Crear una lista para almacenar los DataFrames
df_list = []

# Iterar sobre los archivos y cargar cada archivo CSV en un DataFrame
for archivo_name in archivos:
    if archivo_name.endswith('.csv'):
        archivo_path = os.path.join(carpeta, archivo_name)
        df = spark.read.csv(archivo_path, header=True)
        # Convertir la columna "MISC" a tipo String si es necesario
        if 'MISC' in df.columns:
            df = df.withColumn("MISC", col("MISC").cast("string"))
        df_list.append(df)
# Unir los DataFrames en uno solo
df_re = df_list[0]
for dataframe in df_list[1:]:
    df_re = df_re.union(dataframe)

# Mostrar el DataFrame final
df_re.show()

AnalysisException: [NUM_COLUMNS_MISMATCH] UNION can only be performed on inputs with the same number of columns, but the first input has 9 columns and the third input has 5 columns.;
'Union false, false
:- Relation [IykJMMZgbNcUndwf1q6ieg#2870,2005-02-16T03:23:22.000Z#2871,3zBJUlWtPNoZ0uN83ODbyg#2872,2bXm0SynOfxDzfrdrCyXqg#2873,It's not chicago style pizza but it is still awsome.  A chicago themed restaurant.  Owners used to live in Chi town and now are bringing the feel of the city to the West.  Beer selection rocks!#2874,4.0#2875,06#2876,07#2877,08#2878] csv
:- Relation [XyEpjcexEBlnoBGIo-GImA#2905,2009-06-28T04:28:49.000Z#2906,MUIi5-ElmoGOM9RR5FRTVw#2907,WnClhWcKwtrUD1U0cDhDgg#2908,Not fantastic, but we enjoyed it more than some others in Boise.  They have an interesting selection, but the fish wasn't very good in any of them, and it was rather expensive.  Glad we tried it once, but since there are places with better sushi, we'll probably frequent them instead of Raw.#2909,2.0#2910,